In [16]:
# Import the necessary libraries
from Bio import SeqIO
from transformers import BertTokenizerFast, BertConfig, BertForMaskedLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments, pipeline
import torch
from torch.utils.data import Dataset

In [2]:
# Check if CUDA is available
print("CUDA available:", torch.cuda.is_available())
print("Number of GPUs:", torch.cuda.device_count())

CUDA available: True
Number of GPUs: 1


In [3]:
# Load sequences from the FASTA file
fasta_file = "./data/uniprot_sprot.fasta"
sequences = [str(record.seq) for record in SeqIO.parse(fasta_file, "fasta")]

print(f"Loaded {len(sequences)} sequences")
print("Example:", sequences[0])

Loaded 574627 sequences
Example: MAFSAEDVLKEYDRRRRMEALLLSLYYPNDRKLLDYKEWSPPRVQVECPKAPVEWNNPPSEKGLIVGHFSGIKYKGEKAQASEVDVNKMCCWVSKFKDAMRRYQGIQTCKIPGKVLSDLDAKIKAYNLTVEGVEGFVRYSRVTKQHVAAFLKELRHSKQYENVNLIHYILTDKRVDIQHLEKDLVKDFKALVESAHRMRQGHMINVKYILYQLLKKHGHGPDGPDILTVKTGSKGVLYDDSFRKIYTDLGWKFTPL


In [4]:
# Define the amino acid vocabulary and special tokens for BERT
amino_acids = "ACDEFGHIKLMNPQRSTVWYUOBZX" # standard 20 amino acids + 5 non-standard amino acids (U, O, B, Z, X)
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]  # special tokens for BERT
vocab_dict = {token: idx for idx, token in enumerate(list(amino_acids) + special_tokens)}

# Initialize the BERT tokenizer with the custom vocabulary
tokeniser = BertTokenizerFast(
    vocab=vocab_dict,
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
    do_lower_case=False
)

# Tokenise the sequences
sequences_spaced = [" ".join(seq) for seq in sequences] # add spaces for tokenisation
encodings = tokeniser(sequences_spaced, return_tensors="pt", truncation=True, padding="max_length", max_length=512)
print(encodings["input_ids"].shape)

torch.Size([574627, 512])


In [6]:
# Create a custom dataset class for the tokenised sequences
class ProteinDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __len__(self):
        return self.encodings["input_ids"].shape[0]
    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}

# Create the dataset from encodings
dataset = ProteinDataset(encodings)

In [8]:
# BERT configuration and model initialisation
config = BertConfig(
    vocab_size=len(tokeniser),
    hidden_size=256,
    num_hidden_layers=4,
    num_attention_heads=4,
    max_position_embeddings=512,
    pad_token_id=tokeniser.pad_token_id,
)

model = BertForMaskedLM(config)

In [9]:
# Define the data collator for masked language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokeniser, mlm=True, mlm_probability=0.15
)

In [10]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./mlm_bert",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
    learning_rate=5e-5,
    weight_decay=0.01,
    remove_unused_columns=False,
)

# Initialise the Trainer and start training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator,
)

trainer.train()

Step,Training Loss
100,2.931039
200,2.762342
300,2.727631
400,2.702413
500,2.697768
600,2.694898
700,2.686917
800,2.683466
900,2.680712
1000,2.684854


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

TrainOutput(global_step=71829, training_loss=2.646426249208291, metrics={'train_runtime': 4650.385, 'train_samples_per_second': 123.565, 'train_steps_per_second': 15.446, 'total_flos': 1.3112982108715008e+16, 'train_loss': 2.646426249208291, 'epoch': 1.0})

In [11]:
# Save the trained model and tokeniser
model.save_pretrained("./mlm_bert")
tokeniser.save_pretrained("./mlm_bert")

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

('./mlm_bert\\tokenizer_config.json', './mlm_bert\\tokenizer.json')

In [ ]:
# Load the trained model and tokeniser for inference
model = BertForMaskedLM.from_pretrained("./mlm_bert")
tokeniser = BertTokenizerFast.from_pretrained("./mlm_bert")

Loading weights:   0%|          | 0/74 [00:00<?, ?it/s]

In [24]:
# Test the model with a sample sequence containing a masked token
sample_sequence = "M A F S A E D V L K E [MASK] D R R R R M E A L L L S"
unmasker = pipeline("fill-mask", model=model, tokenizer=tokeniser)
unmasker(sample_sequence)

[{'score': 0.15677320957183838,
  'token': 14,
  'token_str': 'R',
  'sequence': 'M A F S A E D V L K E R D R R R R M E A L L L S'},
 {'score': 0.1200999841094017,
  'token': 3,
  'token_str': 'E',
  'sequence': 'M A F S A E D V L K E E D R R R R M E A L L L S'},
 {'score': 0.10872702300548553,
  'token': 9,
  'token_str': 'L',
  'sequence': 'M A F S A E D V L K E L D R R R R M E A L L L S'},
 {'score': 0.08843398839235306,
  'token': 0,
  'token_str': 'A',
  'sequence': 'M A F S A E D V L K E A D R R R R M E A L L L S'},
 {'score': 0.07427417486906052,
  'token': 8,
  'token_str': 'K',
  'sequence': 'M A F S A E D V L K E K D R R R R M E A L L L S'}]